In [ ]:
import sys
# add parent directory and its parent to sys.path so that python finds the modules
sys.path.append('..')
sys.path.append('../..')

from datetime import datetime

# import pandas as pd
# from sklearn.cluster import KMeans
# from sklearn.preprocessing import MinMaxScaler
# import numpy as np
# import seaborn as sns
import matplotlib.pyplot as plt

from db_utils import get_rect_to_rect_data
# from cluster_utils import cluster_by_max_projection_and_distance
from cluster_utils import cluster_and_plot_for_intersection

In [ ]:
plt.rcParams.update({
    "figure.facecolor":  'white', 
    "axes.facecolor":    'white', 
    "savefig.facecolor": 'white', 
})

In [ ]:
# use data until ..
end_date_str = '2023-01-01 00:00:00'
end_date = datetime.strptime(end_date_str, '%Y-%m-%d %H:%M:%S')

In [ ]:
# files you want to exclude, e.g. because they use the pedestrian way
files_to_exclude = ['VM2_-2112701535', 'VM2_-217686115','VM2_-1247665811','VM2_-104300786', 'VM2_330973206', 'VM2_1100569031', 'VM2_421371629', 'VM2_421371629', 'VM2_1476499235', 'VM2_-1451152685', 'VM2_-1523872256']

In [ ]:
start_end_coords = {
    'Otto-Braun-Str. onto Karl-Marx-Allee (north to east)':
    (
        (13.416448,52.522311,13.416889,52.522671),
        (13.416743,52.521404,13.417205,52.521852)
    ),
     'Karl-Marx-Allee onto Alexanderstr. (east to south)':
    (
        (13.41673,52.5219,13.417469,52.522088),
        (13.416021,52.521813,13.416321,52.52203)
        # (13.415876,52.521774,13.416321,52.52203) # larger endbox
    ),
    'Alexanderstr. onto Alexanderstr. (south to west)':
    (
        (13.416169,52.521508,13.416513,52.521832),
        (13.41591,52.522275,13.416421,52.522569)
    ),    
    'Alexanderstr. onto Otto-Braun-Str. (west to north)':
    (
        (13.415638,52.522086,13.415982,52.522299),
        (13.41685,52.52219,13.417194,52.522403)
    )

}

In [ ]:
cluster_and_plot_for_intersection(start_end_coords, figsize_paths=(8,8))